# Solutions to exercises in the `ray-core` Lessons

## Exercise 1 in 02-DataParallelism-Part1

You were asked to convert the regular Python code to Ray code. Here are the three cells appropriately modified.

First, we need the appropriate imports and `ray.init()`.

In [ ]:
import time, ray
ray.init()

In [ ]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

In [ ]:
start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = ray.get(ids)
duration = time.time() - start

In [ ]:
assert squares == [0, 1, 4, 9]
# should fail until the code modifications are made:
assert duration < 4.1, f'duration = {duration}' 